In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## ■ Similarity Filtering

In [ ]:
# !pip3 install -U spaCy
# !python -m spaCy download en_core_web_lg

__* en_core_web_lg__
> 영어를 위한 훈련된 파이프라인입니다. CPU에 최적화되어 있으며 ok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer와 같은 구성 요소를 포함합니다. 파일 크기는 en_core_web_md와 비교하여 741MB입니다. Spacy에서 제공하는 가장 큰 영어 모델입니다.

### - [TRY 1] spaCy와 similarity 활용

In [ ]:
import ast
import spacy

# spacy의 자연어 처리 기능을 사용하여 키워드 간의 유사도를 계산
# 입력값으로 두 개의 키워드 문자열과 spacy 객체를 받으며, 출력값은 두 문장 간의 유사도로 계산된 float
def similarity_between_keywords(keyword1, keyword2, nlp):
    doc1 = nlp(keyword1)
    doc2 = nlp(keyword2)
    
    return doc1.similarity(doc2)

# 주어진 키워드 목록을 순회하면서 유사한 구문을 필터링
def filter_similar_keywords(keywords, threshold=0.7):
    nlp = spacy.load("en_core_web_lg")
    filtered_keywords = []

    #  순서대로 키워드를 검토하면서 기존 대표 키워드들(filtered_keywords)과 비교하여 유사도가 threshold보다 높은 키워드가 없으면 그 키워드를 대표 키워드 목록에 추가
    for keyword in keywords:
        if not any([similarity_between_keywords(keyword, existing_keyword, nlp) > threshold for existing_keyword in filtered_keywords]):
            filtered_keywords.append(keyword)
    
    return filtered_keywords

keywords = "['treated reason pale', 'grassy aroma taste', 'blond grassy aroma', 'malt low bitterness', 'treated', 'grassy aroma', 'low bitterness', 'straw', 'malt', 'clear blond']"

# filtered_keywords = filter_similar_keywords(keywords) # 키워드가 단순 나열된 형태로 저장되어 있을 경우
filtered_keywords = filter_similar_keywords(ast.literal_eval(keywords)) # 키워드가 리스트형태의 문자열로 저장되어 있을 경우 (Type: String to List) 

print(filtered_keywords)


['treated reason pale', 'grassy aroma taste', 'malt low bitterness', 'straw', 'malt', 'clear blond']


### - [TRY 2] spaCy와 밀도 기반 클러스터링(DBSCAN) 활용

In [3]:
import numpy as np
import spacy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

def select_representative_keyword(cluster_keywords, keyword_vectors):
    if len(cluster_keywords) == 1:
        return cluster_keywords.pop()
    
    centroid = sum(keyword_vectors) / len(keyword_vectors)
    closest_keyword_index = min(range(len(keyword_vectors)), key=lambda i: np.linalg.norm(centroid - keyword_vectors[i]))

    return cluster_keywords[closest_keyword_index]

def merge_similar_keywords_v3(keywords, eps=0.2, min_samples=1):
    nlp = spacy.load('en_core_web_sm')

    # 키워드 벡터화
    keyword_vectors = [nlp(keyword).vector for keyword in keywords]

    # 코사인 유사도를 사용해 DBSCAN 클러스터링
    db = DBSCAN(eps=eps, min_samples=min_samples, metric="precomputed")
    clusters = db.fit_predict(cosine_similarity(keyword_vectors))

    # 클러스터에 속한 키워드를 합치기
    merged_keywords = {}
    for index, cluster in enumerate(clusters):
        if cluster not in merged_keywords:
            merged_keywords[cluster] = ([], [])
        
        merged_keywords[cluster][0].append(keywords[index])
        merged_keywords[cluster][1].append(keyword_vectors[index])

    # 클러스터에서 가장 중심에 위치한 키워드 선택
    result = [select_representative_keyword(keywords, keyword_vectors) for keywords, keyword_vectors in merged_keywords.values()]

    return result

keywords = ['treated reason pale', 'grassy aroma taste', 'blond grassy aroma', 'malt low bitterness', 'treated', 'grassy aroma', 'low bitterness', 'straw', 'malt', 'clear blond']
merged_keywords = merge_similar_keywords_v3(keywords)
print(merged_keywords)


['treated reason pale', 'blond grassy aroma']
